# 1) Spatial Data Prediction (GRID point Classification for subareas)

    This code automates the process of predicting classifications for spatial features, leveraging a pre-trained Random Forest model. It reads CSV files containing geographical features and their geometries, performs data preprocessing (including missing value imputation and feature scaling), and uses the model to predict classifications for each spatial feature, finally saving the results to new CSV files.

In [ ]:
import os
import time
import geopandas as gpd
import pandas as pd
import rasterio
import shapely
from rasterio.plot import show
import pickle
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from shapely import wkt

csv_folder = '/path/to/feature_value_extraction_csv/'
result_csv = '/path/to/save/prediction_result_csv/'

start = time.process_time()

csv_files = [file for file in os.listdir(csv_folder) if file.endswith(".csv")]

for csv_file in csv_files:
    csv_path = os.path.join(csv_folder, csv_file)
    print("Processing:", csv_path)
    s = time.process_time()
    try:
        df = pd.read_csv(csv_path)
        grid_gdf = gpd.GeoDataFrame(data=df, 
                                    geometry=df['geometry'].apply(shapely.wkt.loads), 
                                    crs="EPSG:32631")

        grid_features = df.iloc[:, 2 : df.shape[1]]
        inputation = SimpleImputer(missing_values=np.nan, strategy="median")
        grid_features_filled = inputation.fit_transform(grid_features)
        
        filename = "/path/to/RF_model.sav"
        loaded_model = pickle.load(open(filename, "rb"))
        
        result = pd.DataFrame(loaded_model.predict(grid_features_filled))
        
        df1 = pd.merge(df.iloc[:, 1], result, left_index=True, right_index=True)
        df1.columns = ["geometry", "pred_class"]

        output_csv_path = os.path.join(result_csv, f"predicted_{csv_file}")
        df1.to_csv(output_csv_path, sep=",", index=False, header=True)
        
        print("Predictions saved to:", output_csv_path)
        
    except Exception as e:
        print("Error processing:", csv_path)
        print("Error:", e)
    print(f"Processing time in [min]", (time.process_time() - s) / 60)

print("Prediction process completed")
print("Processing time in [min]", (time.process_time() - start) / 60)